In [ ]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
from io import BytesIO
from PIL import Image
from scipy import ndimage
import math
import cv2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})


</script>
"""

def draw(filename='drawing.png', w=280, h=280, line_width=10):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  data_url = data.split(',')[1]
  img_bytes = b64decode(data_url)
  img = Image.open(BytesIO(img_bytes))
  img  = numpy.array(img)
  b,g,r,img = cv2.split(img)

  return img

def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = numpy.round(cols/2.0-cx).astype(int)
    shifty = numpy.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = numpy.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

def score_gray(gray):
    (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    while numpy.sum(gray[0]) == 0:
        gray = gray[1:]

    while numpy.sum(gray[:,0]) == 0:
        gray = numpy.delete(gray,0,1)

    while numpy.sum(gray[-1]) == 0:
        gray = gray[:-1]

    while numpy.sum(gray[:,-1]) == 0:
        gray = numpy.delete(gray,-1,1)
    rows,cols = gray.shape

    if rows > cols:
        factor = 20.0/rows
        rows = 20
        cols = int(round(cols*factor))
        gray = cv2.resize(gray, (cols,rows))
    else:
        factor = 20.0/cols
        cols = 20
        rows = int(round(rows*factor))
        gray = cv2.resize(gray, (cols, rows))

    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    gray = numpy.lib.pad(gray,(rowsPadding,colsPadding),'constant')
    shiftx,shifty = getBestShift(gray)
    shifted = shift(gray,shiftx,shifty)
    gray = shifted

    gray = gray.astype('float32')
    gray /= 255
    gray = gray.reshape(28,28,1)
    gray = numpy.array([gray])

    pred = model.predict(gray)[0]
    max = 0
    index = -1
    for j in range(10):
        if pred[j] > max:
            max = pred[j]
            index = j
    print('Prediction: ', index)
    print('Certainty:  ', pred[index])

    return index, pred[index]

seed=0
numpy.random.seed(seed)
tensorflow.keras.utils.set_random_seed(seed)

# Neuer Abschnitt

In [ ]:
num_class = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_rows, img_cols = x_train[0].shape

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = tensorflow.keras.utils.to_categorical(y_train, num_class)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_class)

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size= (3,3), activation='ReLU', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='ReLU'))
model.add(Dense(num_class,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2360 - accuracy: 0.9320
Epoch 2/3
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0950 - accuracy: 0.9718
Epoch 3/3
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0618 - accuracy: 0.9815


In [ ]:
model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 0.0644 - accuracy: 0.9782


[0.0643952265381813, 0.9782000184059143]

In [ ]:
img = draw()
score_gray(img)

1/1 [==============================] - 0s 16ms/step
Prediction:  8
Certainty:   0.99640465


(8, 0.99640465)